In [1]:
""" Demand data is provided as volumes. Calculate Fluxes.
-------------------------------------------------------------------------------
PCRGLOBWB Data for demand is provided in volumes with units millionm3 
(implicit per pixel/per time step); converting to fluxes (m) and storing to 
earth engine asset. 

Converts demand to flux
Converts supply and discharge to volume and flux

Author: Rutger Hofste
Date: 20180418
Kernel: python35
Docker: rutgerhofste/gisdocker:ubuntu16.04

Args:

    SCRIPT_NAME (string) : Script name
    EE_VERSION (integer) : Earth Engine path version.
    OUTPUT_VERSION (integer) : Output version.
    TESTING (boolean) : Testing.

Returns:

"""

# Input Parameters

SCRIPT_NAME = "Y2018M04D18_RH_Demand_Fluxes_5min_EE_V01"
EE_VERSION = 9

OUTPUT_VERSION = 2 

TESTING = 0

ee_path = "projects/WRI-Aquaduct/PCRGlobWB20V{:02.0f}".format(EE_VERSION)


print("Input ee: " + ee_path +
      "\nOutput ee: " + ee_path )

Input ee: projects/WRI-Aquaduct/PCRGlobWB20V09
Output ee: projects/WRI-Aquaduct/PCRGlobWB20V09


In [2]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2018M04D19 UTC 13:07


'3.5.4 |Anaconda, Inc.| (default, Nov 20 2017, 18:44:38) \n[GCC 7.2.0]'

In [3]:
import subprocess
import pandas as pd
import ee
import re
import aqueduct3

ee.Initialize()

In [4]:

def update_property_script_used(image):
    image = image.set("script_used",SCRIPT_NAME)
    return image

def update_property_output_version(image):
    image = image.set("output_version",OUTPUT_VERSION)
    return image

def get_df_from_ic(ic_input_asset_id):
    """Create a pandas dataframe with the image asset ids from an imagecollection.
    
    
    """
    command = "earthengine ls {}".format(ic_input_asset_id)
    asset_list_bytes = subprocess.check_output(command,shell=True).splitlines()
    asset_list =[x.decode("utf-8")  for x in asset_list_bytes]
    
    df = pd.DataFrame(asset_list)
    df.columns = ["input_image_asset_id"]
    df["input_ic_asset_id"] = ic_input_asset_id
    return df

    
def add_export_parameters_for_export(df,old_unit,new_unit):
    df_out = df.copy()
    df_out["output_ic_asset_id"] = df_out["input_ic_asset_id"].apply(lambda x: re.sub(old_unit,new_unit,x))
    df_out["output_image_asset_id"] = df_out["input_image_asset_id"].apply(lambda x: re.sub(old_unit,new_unit,x))
    df_out["description"] = df_out["output_image_asset_id"].apply(lambda x: (re.split("/",x)[-1])+"_V{:02.0f}".format(OUTPUT_VERSION)) 
    return df_out


def create_image_collections(df):
    """ Create ImageCollections based on export ic in dataframe
    
    finds unique output_ic_asset_id and creates imageCollections.
    
    """
    
    ic_asset_ids = list(df["output_ic_asset_id"].unique())
    
    
    for ic_asset_id in ic_asset_ids:
        command, result = aqueduct3.earthengine.create_imageCollection(ic_asset_id)
        print(command,result)

In [ ]:
old_unit = "millionm3"
new_unit = "m"

sectors = ["PDom","PInd","PIrr","PLiv"]
demand_types = ["WW","WN"]
temporal_resolutions = ["year","month"]

if TESTING:
    sectors = ["PDom"]
    demand_types = ["WW"]
    temporal_resolutions = ["year"]


df = pd.DataFrame()
df_merged = pd.DataFrame()
for sector in sectors:
    for demand_type in demand_types:
        for temporal_resolution in temporal_resolutions:
            print(sector,demand_type,temporal_resolution)
            ic_input_file_name = "global_historical_{}{}_{}_millionm3_5min_1960_2014".format(sector,demand_type,temporal_resolution)
            ic_input_asset_id = "{}/{}".format(ee_path,ic_input_file_name)
            df = get_df_from_ic(ic_input_asset_id)
            df = add_export_parameters_for_export(df,old_unit,new_unit)
            create_image_collections(df)
            
            for index, row in df.iterrows():
                i_volume_millionm3_5min = ee.Image(row["input_image_asset_id"])
                i_flux_m_5min = aqueduct3.earthengine.volume_to_flux_5min_millionm3_m2(i_volume_millionm3_5min)
                i_flux_m_5min = update_property_script_used(i_flux_m_5min)
                i_flux_m_5min = update_property_output_version(i_flux_m_5min)
                
                description = row["description"]
                output_image_asset_id = row["output_image_asset_id"]
                aqueduct3.earthengine.export_image_global_5min(i_flux_m_5min,description,output_image_asset_id)
                print(output_image_asset_id)

PDom WW year
earthengine create collection projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_year_m_5min_1960_2014 b''
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_year_m_5min_1960_2014/global_historical_PDomWW_year_m_5min_1960_2014_I000Y1960M12
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_year_m_5min_1960_2014/global_historical_PDomWW_year_m_5min_1960_2014_I001Y1961M12
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_year_m_5min_1960_2014/global_historical_PDomWW_year_m_5min_1960_2014_I002Y1962M12
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_year_m_5min_1960_2014/global_historical_PDomWW_year_m_5min_1960_2014_I003Y1963M12
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_year_m_5min_1960_2014/global_historical_PDomWW_year_m_5min_1960_2014_I004Y1964M12
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_year_m_5min_1960_2014/global_historical_PDomWW_year_m_5min_1960_2014_I005Y1965M12
proje

projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1960_2014_I001Y1960M02
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1960_2014_I002Y1960M03
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1960_2014_I003Y1960M04
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1960_2014_I004Y1960M05
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1960_2014_I005Y1960M06
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1960_2014_I006Y1960M07
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1

projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1960_2014_I058Y1964M11
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1960_2014_I059Y1964M12
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1960_2014_I060Y1965M01
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1960_2014_I061Y1965M02
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1960_2014_I062Y1965M03
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1960_2014_I063Y1965M04
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1

projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1960_2014_I115Y1969M08
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1960_2014_I116Y1969M09
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1960_2014_I117Y1969M10
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1960_2014_I118Y1969M11
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1960_2014_I119Y1969M12
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1960_2014_I120Y1970M01
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1

projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1960_2014_I172Y1974M05
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1960_2014_I173Y1974M06
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1960_2014_I174Y1974M07
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1960_2014_I175Y1974M08
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1960_2014_I176Y1974M09
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1960_2014_I177Y1974M10
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_m_5min_1960_2014/global_historical_PDomWW_month_m_5min_1

...

projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWW_month_m_5min_1960_2014/global_historical_PIndWW_month_m_5min_1960_2014_I505Y2002M02
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWW_month_m_5min_1960_2014/global_historical_PIndWW_month_m_5min_1960_2014_I506Y2002M03
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWW_month_m_5min_1960_2014/global_historical_PIndWW_month_m_5min_1960_2014_I507Y2002M04
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWW_month_m_5min_1960_2014/global_historical_PIndWW_month_m_5min_1960_2014_I508Y2002M05
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWW_month_m_5min_1960_2014/global_historical_PIndWW_month_m_5min_1960_2014_I509Y2002M06
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWW_month_m_5min_1960_2014/global_historical_PIndWW_month_m_5min_1960_2014_I510Y2002M07
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWW_month_m_5min_1960_2014/global_historical_PIndWW_month_m_5min_1

projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWW_month_m_5min_1960_2014/global_historical_PIndWW_month_m_5min_1960_2014_I562Y2006M11
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWW_month_m_5min_1960_2014/global_historical_PIndWW_month_m_5min_1960_2014_I563Y2006M12
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWW_month_m_5min_1960_2014/global_historical_PIndWW_month_m_5min_1960_2014_I564Y2007M01
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWW_month_m_5min_1960_2014/global_historical_PIndWW_month_m_5min_1960_2014_I565Y2007M02
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWW_month_m_5min_1960_2014/global_historical_PIndWW_month_m_5min_1960_2014_I566Y2007M03
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWW_month_m_5min_1960_2014/global_historical_PIndWW_month_m_5min_1960_2014_I567Y2007M04
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWW_month_m_5min_1960_2014/global_historical_PIndWW_month_m_5min_1

projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWW_month_m_5min_1960_2014/global_historical_PIndWW_month_m_5min_1960_2014_I619Y2011M08
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWW_month_m_5min_1960_2014/global_historical_PIndWW_month_m_5min_1960_2014_I620Y2011M09
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWW_month_m_5min_1960_2014/global_historical_PIndWW_month_m_5min_1960_2014_I621Y2011M10
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWW_month_m_5min_1960_2014/global_historical_PIndWW_month_m_5min_1960_2014_I622Y2011M11
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWW_month_m_5min_1960_2014/global_historical_PIndWW_month_m_5min_1960_2014_I623Y2011M12
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWW_month_m_5min_1960_2014/global_historical_PIndWW_month_m_5min_1960_2014_I624Y2012M01
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWW_month_m_5min_1960_2014/global_historical_PIndWW_month_m_5min_1

projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWN_year_m_5min_1960_2014/global_historical_PIndWN_year_m_5min_1960_2014_I015Y1975M12
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWN_year_m_5min_1960_2014/global_historical_PIndWN_year_m_5min_1960_2014_I016Y1976M12
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWN_year_m_5min_1960_2014/global_historical_PIndWN_year_m_5min_1960_2014_I017Y1977M12
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWN_year_m_5min_1960_2014/global_historical_PIndWN_year_m_5min_1960_2014_I018Y1978M12
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWN_year_m_5min_1960_2014/global_historical_PIndWN_year_m_5min_1960_2014_I019Y1979M12
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWN_year_m_5min_1960_2014/global_historical_PIndWN_year_m_5min_1960_2014_I020Y1980M12
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWN_year_m_5min_1960_2014/global_historical_PIndWN_year_m_5min_1960_2014_I021Y

projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWN_month_m_5min_1960_2014/global_historical_PIndWN_month_m_5min_1960_2014_I016Y1961M05


In [ ]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

Previous runs:  
